In [ ]:
import sys
import re
from reprowd.crowdcontext import CrowdContext
from reprowd.presenter.base import BasePresenter
sys.path.append('..')

# Defining constants

In [ ]:
input_image_path = '../../img/lena.bmp'
n_x = 10
n_y = 10
input_splits_folder = '../../out_img/'
n_assigns = 3
output_stitched_folder = '../../stitched/'
output_blended_image_path = '../../blended.jpg'
ftp_divs_folder = 'img_divs'

# Project Attributes
project_long_name = 'CSF_long'
project_short_name = 'CSF_short'
project_description = 'Crowdsourced Sketch Filter'
presenter_question = 'Replicate the original image as closely as you can.'

# Splitting and uploading the input image

The input image is splitted into multiple regions and saved in a local folder by the splitter script

In [ ]:
from splitter import split_image
img_split_paths = split_image(input_image_path, n_x, n_y, input_splits_folder)

The uploader script uploads all regions from the local folder into an FTP server

In [ ]:
from uploader import upload_images
img_split_urls = upload_images(img_split_paths, ftp_divs_folder)

# Querying the crowd

The context is loaded from a previously saved state, or a new one is created

In [ ]:
cc = CrowdContext(local_db="sketch.db")

The urls of the uploaded images are sent into the context database

In [ ]:
crowd_data = cc.CrowdData(img_split_urls, 'image_regions')

Our custom made presenter template is set as the current one

In [ ]:
my_presenter = BasePresenter()
my_presenter.set_name(project_long_name)
my_presenter.set_short_name(project_short_name)
my_presenter.set_description(project_description)
my_presenter.set_question(presenter_question)

# Set the name of the project in the template
html_text = None
with open('presenter.html', 'rw') as presenter_file:
    html_text = presenter_file.read()
    html_text = re.sub("pybossa\.userProgress\(\'([\S]+)\'\)",
                       "pybossa.userProgress('" + project_short_name + "')",
                       html_text)
    html_text = re.sub("pybossa\.run\(\'([\S]+)\'\)",
                       "pybossa.run('" + project_short_name + "')",
                       html_text)

# Set the template as the presenter
my_presenter.set_template(html_text)
crowd_data.set_presenter(my_presenter, lambda obj: {'url_m': obj})

The tasks are published with a predefined number of assignments per task

In [ ]:
r = crowd_data.publish_task(n_assignments=n_assigns)

# Gathering results

The resulting images are stored in a big dictionary with more information than we are going to use

In [ ]:
crowd_data = crowd_data.get_result()

We traverse *crowd_data* gathering only the information that will be useful in an organized manner

The images are stored in a base64 string, so during the traversal we are also decode those strings back into jpeg images.

The result is *gathered_results* array, which is an array of dicts containing the file name and one image per assignment for that filename.

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def gather_images():
    results = []
    for i in crowd_data.data['id']:
        result_obj = {}
        m = re.search('([0-9]+_[0-9]+_[0-9]+_[0-9]+).[a-zA-Z]+', crowd_data.data['object'][i])
        result_obj['file'] = m.group(1)
        result_obj['img'] = []
        for a in crowd_data.data['result'][i]['assignments']:
            im = Image.open(BytesIO(base64.b64decode(a['worker_response'][23:])))
            result_obj['img'].append(im)
        results.append(result_obj)
    return results

gathered_results = gather_images()
            

The stitcher script stitches the images back together and saves them in a local folder

In [ ]:
from stitcher import stitch_images_from_object
stitch_images_from_object(input_image_path, gathered_results, output_stitched_folder)

The diferrent versions of the original image are blended, in order to create an average of all stitched images.

In [ ]:
from blender import blend_images_from_folder
blend_images_from_folder(output_stitched_folder, output_blended_image_path)